In [3]:
# Настройте путь к PLY здесь (измените при необходимости):
from pathlib import Path
PLY_PATH = Path('mmdetection3d/outputs/car_pcl.ply')  # по умолчанию
OUT_DIR = Path('mmdetection3d/outputs')
OUT_DIR.mkdir(parents=True, exist_ok=True)
print('PLY путь:', PLY_PATH.resolve())

PLY путь: /workspaces/mmdet3d/mmdetection3d/outputs/car_pcl.ply


In [4]:
# Загрузка PLY с помощью open3d и конвертация в numpy
import open3d as o3d
import numpy as np

if not PLY_PATH.exists():
    print(f'Файл не найден: {PLY_PATH.resolve()}')
else:
    pcd = o3d.io.read_point_cloud(str(PLY_PATH))
    points = np.asarray(pcd.points)
    print('Загружено точек:', points.shape)
    # опционально: цвета
    colors = None
    if pcd.has_colors():
        colors = np.asarray(pcd.colors)
        print('Цвета найдены')

Загружено точек: (17238, 3)
Цвета найдены


In [5]:
# Визуализация через open3d (откроет отдельное окно, если поддерживается)
try:
    o3d.visualization.draw_geometries([pcd], window_name='Open3D PointCloud', width=800, height=600)
except Exception as e:
    print('Open3D визуализация недоступна:', e)

[Open3D WARNING] GLFW Error: Failed to detect any supported platform
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


error: XDG_RUNTIME_DIR is invalid or not set in the environment.


In [ ]:
# Быстрая 3D-визуализация (matplotlib).
# Для больших облаков точек рекомендуется уменьшить размер выборки.
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Параметры визуализации:
MAX_POINTS = 100000  # уменьшить для ускорения

try:
    n = points.shape[0]
    if n > MAX_POINTS:
        idx = np.random.choice(n, MAX_POINTS, replace=False)
        sample = points[idx]
        sample_colors = colors[idx] if (colors is not None) else None
    else:
        sample = points
        sample_colors = colors

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Цвет по высоте (Z)
    c = sample[:, 2]
    p = ax.scatter(sample[:, 0], sample[:, 1], sample[:, 2], c=c, cmap='viridis', s=1, alpha=0.7)
    fig.colorbar(p, ax=ax, label='Z (height)')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Point cloud preview (matplotlib)')
    plt.show()
except NameError:
    print('Сначала выполните предыдущую ячейку для загрузки PLY')

In [ ]:
# Сохранение PNG превью и копии PLY
import os
PNG_OUT = OUT_DIR / 'notebook_visualization.png'
PLY_OUT = OUT_DIR / 'point_cloud_notebook_copy.ply'

try:
    # Сохранить последнюю фигуру как PNG если она есть
    fig.savefig(str(PNG_OUT), dpi=150, bbox_inches='tight')
    print('PNG сохранён:', PNG_OUT)
except Exception as e:
    print('Не удалось сохранить PNG:', e)

try:
    o3d.io.write_point_cloud(str(PLY_OUT), pcd)
    print('PLY копия сохранена:', PLY_OUT)
except Exception as e:
    print('Не удалось сохранить PLY:', e)

## Дополнительно (опционально)
Если хотите более интерактивный просмотр внутри Jupyter (вращение/интерактивность), можно установить `pyvista` или использовать `plotly` для `scatter3d`. Пример установки:

```
pip install pyvista
pip install plotly
```

И пример вызова PyVista: `pv.plot(pv.PolyData(points), point_size=1)`  (после `import pyvista as pv`).